### Problem: <br>
You have a website where  you are selling books(say 7). For  the website you have to create a  search engine where the visitor  will a search term and one of the  book details will be shown. You  have the following dataset of the  title of the book., description,  keywords(tags) and price of the book.

We have to create a search engine with keywords of the books.

#### Solution:
For this  problem we will recieve the user input in normal  language, so we need to use NLP and then we need  to classify the search result into one book so we  need use a classifier too.

We have to create another dataset which  will extract all the words from the keywords for  each book and then form unique combinations with  them and then we can apply CountVectorizer and  TfidfTransformer's to ready them for our  classifier to understand and then we will train  our classifier that will be a decision tree  classifier and then we can pass the search terms to it


In [4]:
pip install pandas

  Using cached pandas-1.5.2-cp310-cp310-macosx_11_0_arm64.whl (10.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 6.1 MB/s eta 0:00:0000:0100:01
  Using cached numpy-1.24.1-cp310-cp310-macosx_11_0_arm64.whl (13.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn

  Using cached scikit_learn-1.2.0-cp310-cp310-macosx_12_0_arm64.whl (8.3 MB)
  Using cached scipy-1.10.0-cp310-cp310-macosx_12_0_arm64.whl (28.8 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 

In [4]:
dt=pd.read_csv('data/books_metadata.csv')
dt

,Title,Description,Keywords,Price
0,It's Python,A beginners guidebook to programming with Pyth...,"python for beginners, python programming",$3.00
1,Data Science with Python,Data Science with Python is a book created to ...,"data science, python",$3.09
2,Machine learning with Python,Do you know all the automatic recommendations ...,"machine learning, python, regression, classifi...",$3.19
3,Artificial Intelligence with Python,Learn the idea behind how to create intelligen...,"ai, python, artificial intelligence",$3.50
4,Data analysis with Python,Learn how to perform different types of analyt...,"data analysis, pandas, data processing, python",$3.00
5,Data visualization with Python,"Looking at raw data is boring, present it in t...","data visualization, python, matplotlib",$3.00
6,OOP with Python,Object Oriented Programming with Python. Learn...,"object oreiented programming, python",$3.00


#### Extract the keywords

In [5]:
from nltk import word_tokenize 
Kys = dt['Keywords'].values  
b1_kys = word_tokenize(Kys[0])  # Keywords for book 2
b1_kys.remove(',')  
b1_kys 

['python', 'for', 'beginners', 'python', 'programming']

#### Creating Unique combinations

In [7]:
pip install python-constraint

  Preparing metadata (setup.py) ... done
  DEPRECATION: python-constraint is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for python-constraint ... done
Note: you may need to restart the kernel to use updated packages.


In [7]:
import constraint  
#Creating Combinations  
com = constraint.Problem()  
com.addVariables(["a","b"], b1_kys)  
com.addConstraint(constraint.AllDifferentConstraint()) 

In [8]:
wrds = []  
for ele in com.getSolutions():  
    wrds.append(f"{ele['a']} "+ f"{ele['b']}")  
wrds 
 

['programming python',
 'programming beginners',
 'programming for',
 'programming python',
 'python programming',
 'python beginners',
 'python for',
 'beginners python',
 'beginners programming',
 'beginners python',
 'beginners for',
 'for beginners',
 'for python',
 'for programming',
 'for python',
 'python for',
 'python beginners',
 'python programming']

In [10]:
# Extracting the keywords and creating keywords combinations for all books

Titles = dt['Title'].values  
Kys = dt['Keywords'].values  
keywrds = {'B1': [], 'B2': [], 'B3': [], 'B4': [],  'B5': [], 'B6': [], 'B7': []}  
kys = []  
wrds = []  
for i in range(7):  
    kys.clear()  
    wrds.clear()  
    kys = word_tokenize(Kys[i])  
    while (kys.count(',')):  
        kys.remove(',')  
        com = constraint.Problem()  
        com.addVariables(["a","b"],kys)  
        com.addConstraint(  constraint.AllDifferentConstraint())  
        for ele in com.getSolutions():  
            wrds.append(f"{ele['a']} "+ f"{ele['b']}")  
        for k in wrds:  
            keywrds[f"B{i+1}"].append(k) 



In [12]:
# 2-D list with 1-D list with elements as  the keywords combination and the book label i.e.  Bl, B2, B7.
Dt = []  
for bk in list(keywrds.keys()):  
    for ks in keywrds[bk]:  
        Dt.append([ks,bk]) 

In [13]:
import numpy  
Dt = numpy.array(Dt)  
KYS = pd.DataFrame({'Keywords':Dt[:,0],'Book':Dt[:,1]})  
KYS 
 

,Keywords,Book
0,programming python,B1
1,programming beginners,B1
2,programming for,B1
3,programming python,B1
4,python programming,B1
...,...,...
2199,oreiented python,B7
2200,oreiented object,B7
2201,object oreiented,B7
2202,object programming,B7


In [16]:
inp = Dt[:,0]  
Y = Dt[:,1]  
enc = CountVectorizer()  
enc.fit(inp)  
e_inp = enc.transform(inp)  
tf = TfidfTransformer()  
tf.fit(e_inp)  
X = tf.transform(e_inp)  


In [17]:
CModel=DecisionTreeClassifier()
CModel.fit(X,Y)

DecisionTreeClassifier()

In [24]:
def search_book():  
    query = input('Search:')  
    enc_q = enc.transform([query])  
    tf_q = tf.transform(enc_q)  
    bk = CModel.predict(tf_q)  
    i = {'B1': 0, 'B2': 1, 'B3': 2, 'B4': 3,  'B5': 4, 'B6': 5, 'B7': 6}  
    title = dt.values[i[bk[0]],0]  
    desc = dt.values[i[bk[0]],1]  
    price = dt.values[i[bk[0]],3]  
    result = f"""  
    Query:
    {query}
    
    Tittle:  
    {title}  
    
    Description:  
    {desc}  
    
    Price:
    {price}"""  
    print(result) 



In [25]:
search_book()

  
    Query:
    data cleaning
    
    Tittle:  
    Data analysis with Python  
    
    Description:  
    Learn how to perform different types of analytics on data like central tendency, informations gain, feature extraction, Label Encoding, Scaling, Standard deviations, interquartile range, etc. with Python.  
    
    Price:
    $3.00


In [26]:
search_book()

  
    Query:
    book for python
    
    Tittle:  
    It's Python  
    
    Description:  
    A beginners guidebook to programming with Python. We will learn about different data-types in python, decision constrol using if statements, creating loops and how to  iterate over data types.  
    
    Price:
    $3.00


In [27]:
pip install gTTs

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install Pyaudio


Note: you may need to restart the kernel to use updated packages.


In [29]:
def search_book(Type='text'):  
    import speech_recognition  
    from gtts import gTTS  
    import IPython  
    if Type == 'text':  
        query = input('Search:')  
    elif Type == 'voice':  
        voice = speech_recognition.Recognizer()  
        with speech_recognition.Microphone() as a:  
            voice.adjust_for_ambient_noise(a)  
            print("Your Query:")  
            aud = voice.listen(a)  
        query = voice.recognize_google(aud)  
    enc_q = enc.transform([query])  
    tf_q = tf.transform(enc_q)  
    bk = CModel.predict(tf_q)  
    i = {'B1': 0, 'B2': 1, 'B3': 2, 'B4': 3,  'B5': 4, 'B6': 5, 'B7': 6}  
    title = dt.values[i[bk[0]],0]  
    desc = dt.values[i[bk[0]],1]  
    price = dt.values[i[bk[0]],3]  
    result = f"""  
    Query:
    {query}
    
    Tittle:  
    {title}  
    
    Description:  
    {desc}  
    
    Price:
    {price}"""  
    print(result) 
    
    spkr = gTTS(text = result, lang = 'en')  
    spkr.save("voice.mp3")  
    print(result)  
    IPython.display.Audio("voice.mp3",  autoplay=True) 


In [37]:
search_book(Type='voice')

Your Query:
result2:
{   'alternative': [   {   'confidence': 0.95467669,
                           'transcript': 'data cleaning'},
                       {'transcript': 'data-cleaning'}],
    'final': True}
  
    Query:
    data cleaning
    
    Tittle:  
    Data analysis with Python  
    
    Description:  
    Learn how to perform different types of analytics on data like central tendency, informations gain, feature extraction, Label Encoding, Scaling, Standard deviations, interquartile range, etc. with Python.  
    
    Price:
    $3.00
  
    Query:
    data cleaning
    
    Tittle:  
    Data analysis with Python  
    
    Description:  
    Learn how to perform different types of analytics on data like central tendency, informations gain, feature extraction, Label Encoding, Scaling, Standard deviations, interquartile range, etc. with Python.  
    
    Price:
    $3.00
